In [ ]:
! pip install geopandas
! pip install pandas
! pip install pygeos
! pip install boto
! pip install s3fs
! pip install shapely
! pip install gcsfs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import time
from itertools import groupby
import geopandas as gpd

In [ ]:
# READ AND CORRECT LINK TABLE RESULTS
link_table_5fleets = pd.read_csv('s3://beam-outputs/pilates-outputs/sfbay_5fleets_100price_1000fleet_20230209/postprocessOutputs/year-2020-iteration-3/linkTable.csv.gz', nrows = None)

link_table_bs      = pd.read_csv('s3://beam-outputs/pilates-outputs/sfbay_baseline_20230209/postprocessOutputs/year-2020-iteration-3/linkTable.csv.gz', nrows = None)

link_table_bs = link_table_bs.fillna(0)
bgids = []
wrong = 0
for bg in link_table_bs['bgid']:
    bgids.append('0'+str(int(bg)))
    if bg ==0:
        wrong +=1
print('wrong',wrong)
link_table_bs['bgid'] = bgids

link_table_5fleets = link_table_5fleets.fillna(0)
bgids = []
wrong = 0
for bg in link_table_5fleets['bgid']:
    bgids.append('0'+str(int(bg)))
    if bg ==0:
        wrong +=1
print('wrong',wrong)
link_table_5fleets['bgid'] = bgids

#Filter volume higher than 5
link_table_bs_volume = link_table_bs[link_table_bs['volume']>5]
link_table_5fleets_volume = link_table_5fleets[link_table_5fleets['volume']>5]

In [ ]:
# AGGREGATE SPEEDS BY BLOCK ID
link_table_bs_grouped = link_table_bs_volume.groupby(['bgid']).apply(lambda x: np.mean(list(x.length/x.traveltime)))
link_table_5fleets_grouped = link_table_5fleets_volume.groupby(['bgid']).apply(lambda x: np.mean(list(x.length/x.traveltime)))
delta_speeds = {}
for bgid in link_table_bs_grouped.keys():
    try:
        delta_speeds[bgid] = link_table_5fleets_grouped[bgid] - link_table_bs_grouped[bgid] 
    except:
        print(bgid)
speeds_diff = pd.DataFrame()
speeds_diff['speeds_diff'] = delta_speeds.values()
speeds_diff['bgid'] = delta_speeds.keys()
speeds_diff.to_csv('outputs/diff_speeds_bgid.csv')

In [ ]:
# AGGREGATE SPEEDS BY ZIP CODE
link_table_bs_grouped = link_table_bs_volume.groupby(['bgid']).apply(lambda x: np.mean(list(x.length/x.traveltime)))
link_table_5fleets_grouped = link_table_5fleets_volume.groupby(['bgid']).apply(lambda x: np.mean(list(x.length/x.traveltime)))
delta_speeds = {}
for bgid in link_table_bs_grouped.keys():
    try:
        delta_speeds[bgid] = link_table_5fleets_grouped[bgid] - link_table_bs_grouped[bgid] 
    except:
        print(bgid)
speeds_diff = pd.DataFrame()
speeds_diff['speeds_diff'] = delta_speeds.values()
speeds_diff['bgid'] = delta_speeds.keys()
speeds_diff.to_csv('outputs/diff_speeds_bgid.csv')

In [ ]:
link_table_bs

In [ ]:
network     = pd.read_csv('s3://beam-outputs/pilates-outputs/sfbay_baseline_20230209/beam/year-2020-iteration-3/network.csv.gz', nrows = None)
network.to_csv('outputs/network.csv')

In [ ]:
network

In [ ]:
link_table_bs

In [ ]:
[list(delta_speeds.values()),list(delta_speeds.keys())]